<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=c1c45f00d57ab9f557890f317a422a0af0cdfd0b434fcf4a3783c0385b9b8757
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.0
    Uninstalling protobuf-6.33.0:
      Successfully uninstalled protobuf-6.33.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-12 09:30:18
-------------------
qualified stocks: 84
with latest results: 24
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  5.59 L
Current:  1.44 C
-------------------
Today PnL: 70.23 K (0.49%)
Current PnL: -21.61 L (-14.16%)
CY Booked + Current PnL: -8.12 L (-5.32%)
-------------------
Total profit:  1.59 L
Total loss:  -23.19 L
-------------------
Total Booked + Current PnL: 18.86 L (15.02%)
Total Booked PnL: 40.47 L (32.22%)
Curr Year Booked PnL: 13.49 L (9.38%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.30 C
Est FTT PnL: 86.27 L (59.99%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.86%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.00,101.92,51.0,M-SC,2.55,86964.0,-13813.0,13906.0,0.33,-13.71,15.99,0.09,245.0,-0.99,0.63,14.25,OX40N,NTT,DURABLES
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.28,88500.0,-2178.0,17089.0,-0.17,-2.40,19.31,16.44,101.0,-0.13,0.64,17.29,X40,ATH,PHARMA
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,-0.21,-0.80,11.33,10.44,4.0,-0.07,1.44,4.72,X40,NTT,FMCG
33,ICICIGI,2252.93,-13.61,55.0,X-MC,7.53,186763.0,12443.0,22766.0,0.61,7.14,12.19,20.19,91.0,0.55,1.35,23.33,X40,ATH,INSURANCE
50,NESTLEIND,1377.00,-3.90,58.0,X-LC,8.51,295659.0,30233.0,26550.0,-0.36,11.39,8.98,21.40,11.0,1.14,2.14,18.79,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,ROUTE,2227.21,-57.43,35.0,H-SC,23.26,70023.0,-63043.0,154926.0,3.17,-47.38,221.25,69.05,118.0,-0.41,0.51,3.17,SR,ATH,IT
75,TRIDENT,48.00,-8.30,57.0,M-SC,2.07,71383.0,-20872.0,47177.0,2.74,-22.62,66.09,28.51,224.0,-0.44,0.52,22.15,XR,NTT,TEXTILES
38,INDUSINDBK,1800.00,292.16,75.0,L-MC,7.15,48128.0,-29078.0,54471.0,2.21,-37.66,113.18,32.89,258.0,-0.53,0.35,32.54,XR,NTT,BANKS
58,REPCOHOME,880.00,-53.06,65.0,H-SC,9.10,270993.0,-15512.0,272836.0,2.15,-5.41,100.68,89.82,134.0,-0.06,1.96,43.38,XY24,NTT,FINANCE
19,COFFEEDAY,80.00,-47.69,46.0,L-SC,26.29,78221.0,-35328.0,75382.0,2.11,-31.11,96.37,35.27,268.0,-0.47,0.57,91.45,XR,NTT,HOTELS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,INDIGOPNTS,1408.00,124.89,74.0,M-SC,4.47,147349.0,-27210.0,27245.0,-1.19,-15.59,18.49,0.02,221.0,-1.00,1.07,28.17,OX40N,NTT,PAINTS
80,VBL,671.64,-17.63,50.0,X-LC,5.66,296861.0,-18981.0,131658.0,-1.13,-6.01,44.35,35.67,5.0,-0.14,2.15,7.01,X40N,ATH,FMCG
8,ATULAUTO,844.00,3554.05,51.0,M-SC,4.00,164220.0,-32912.0,122738.0,-0.96,-16.70,74.74,45.57,236.0,-0.27,1.19,16.43,XY24,NTT,AUTO
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
9,AWL,485.00,-63.48,55.0,X-MC,2.43,259856.0,-41895.0,203311.0,-0.68,-13.88,78.24,53.50,116.0,-0.21,1.88,15.42,XY24,NTT,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,WHIRLPOOL,2270.0,-44.83,38.0,M-SC,8.89,114374.0,-424.0,94461.0,0.43,-0.37,82.59,81.92,223.0,-0.00,0.83,35.40,XR,NTT,DURABLES
56,RECLTD,446.0,42.86,47.0,M-LC,4.29,201602.0,-1568.0,43707.0,1.24,-0.77,21.68,20.74,55.0,-0.04,1.46,6.04,XY25,NTT,FINANCE
83,WIPRO,420.0,-13.79,54.0,M-LC,5.74,151726.0,781.0,108666.0,1.25,0.52,71.62,72.51,53.0,0.01,1.10,6.57,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TTKPRESTIG,770.0,101.92,51.0,M-SC,2.55,86964.0,-13813.0,13906.0,0.33,-13.71,15.99,0.09,245.0,-0.99,0.63,14.25,OX40N,NTT,DURABLES
37,INDIGOPNTS,1408.0,124.89,74.0,M-SC,4.47,147349.0,-27210.0,27245.0,-1.19,-15.59,18.49,0.02,221.0,-1.00,1.07,28.17,OX40N,NTT,PAINTS
45,KANSAINER,340.0,-68.34,38.0,H-SC,2.28,218745.0,-50922.0,87257.0,-0.33,-18.88,39.89,13.47,138.0,-0.58,1.58,11.23,XY24,NTT,PAINTS
65,SIS,528.0,2014.51,46.0,H-SC,2.24,85336.0,-25696.0,49307.0,0.19,-23.14,57.78,21.26,156.0,-0.52,0.62,15.30,OX40N,NTT,MISC
42,JCHAC,2282.0,19986.96,65.0,M-SC,1.52,98151.0,-29654.0,29642.0,0.20,-23.20,30.20,-0.01,232.0,-1.00,0.71,11.14,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151726.0,781.0,108666.0,1.25,0.52,71.62,72.51,53.0,0.01,1.10,6.57,XR,NTT,IT
36,INDIAMART,4810.62,-53.52,52.0,H-SC,4.86,129871.0,6535.0,125092.0,-0.07,5.30,96.32,106.72,119.0,0.05,0.94,29.92,AR,ATH,MISC
60,SAMMAANCAP,326.00,-192.05,54.0,M-SC,35.60,159381.0,10161.0,134023.0,0.41,6.81,84.09,96.62,208.0,0.08,1.15,72.48,XY25,NTT,FINANCE


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,INDIAMART,4810.62,-53.52,52.0,H-SC,4.86,129871.0,6535.0,125092.0,-0.07,5.30,96.32,106.72,119.0,0.05,0.94,29.92,AR,ATH,MISC
83,WIPRO,420.00,-13.79,54.0,M-LC,5.74,151726.0,781.0,108666.0,1.25,0.52,71.62,72.51,53.0,0.01,1.10,6.57,XR,NTT,IT
82,WHIRLPOOL,2270.00,-44.83,38.0,M-SC,8.89,114374.0,-424.0,94461.0,0.43,-0.37,82.59,81.92,223.0,-0.00,0.83,35.40,XR,NTT,DURABLES
46,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122548.0,-2749.0,61678.0,0.41,-2.19,50.33,47.03,141.0,-0.04,0.89,53.44,MH,ATH,POWER
35,IEX,219.00,-33.86,46.0,H-SC,15.87,189139.0,-10815.0,108036.0,0.03,-5.41,57.12,48.63,115.0,-0.10,1.37,5.52,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
72,TATAMOTORS,1065.00,-53.17,24.0,X-LC,37.92,147959.0,-128564.0,238643.0,-0.69,-46.49,161.29,39.81,54.0,-0.54,1.07,4.29,XY24,NTT,AUTO
25,GILLETTE,11206.78,-17.64,26.0,X-MC,5.73,253350.0,-546.0,82845.0,1.94,-0.22,32.70,32.42,105.0,-0.01,1.83,14.85,X40,ATH,FMCG
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.73,87067.0,-41603.0,86902.0,0.20,-32.33,99.81,35.20,219.0,-0.48,0.63,0.20,X40,NTT,FOOTWEAR
53,QUESS,986.00,-52.07,31.0,X-SC,37.15,47852.0,-17154.0,168085.0,0.01,-26.39,351.26,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
44,JSWINFRA,342.00,-23.75,33.0,X-MC,4.69,191498.0,-8849.0,39353.0,0.11,-4.42,20.55,15.23,178.0,-0.22,1.39,26.41,X40N,NTT,REALTY


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,-0.21,-0.80,11.33,10.44,4.0,-0.07,1.44,4.72,X40,NTT,FMCG
52,PGHH,17907.65,-32.13,37.0,X-MC,1.10,196500.0,-4320.0,72116.0,1.41,-2.15,36.70,33.76,80.0,-0.06,1.42,2.93,X40,ATH,FMCG
64,SIEMENS,4671.50,-5.39,40.0,H-LC,1.67,152090.0,-34005.0,81490.0,0.66,-18.27,53.58,25.51,15.0,-0.42,1.10,12.08,AR,ATH,ELECTRICAL
29,HAVELLS,2069.16,-8.87,38.0,X-MC,3.31,304227.0,-23764.0,130300.0,0.35,-7.25,42.83,32.48,92.0,-0.18,2.20,3.35,X40,ATH,ELECTRICAL
77,UNITDSPR,1644.00,-6.81,61.0,X-LC,3.46,244240.0,10096.0,38517.0,0.39,4.31,15.77,20.77,86.0,0.26,1.77,11.53,X40N,NTT,BREWERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-52.07,31.0,X-SC,37.15,47852.0,-17154.0,168085.0,0.01,-26.39,351.26,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.73,87067.0,-41603.0,86902.0,0.20,-32.33,99.81,35.20,219.0,-0.48,0.63,0.20,X40,NTT,FOOTWEAR
1,ABB,7934.00,-42.69,34.0,H-LC,5.00,239472.0,-22147.0,141360.0,0.21,-8.47,59.03,45.57,7.0,-0.16,1.73,1.90,AR,NTT,ELECTRICAL
10,BAJAJHFL,181.50,-15.96,39.0,X-MC,6.32,177284.0,-23222.0,120376.0,0.39,-11.58,67.90,48.45,90.0,-0.19,1.28,2.23,X40N,ATH,FINANCE
51,PAGEIND,51769.93,-28.51,41.0,X-MC,7.65,161560.0,-2180.0,45528.0,0.42,-1.33,28.18,26.47,82.0,-0.05,1.17,2.36,X40,ATH,APPARELS


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-52.07,31.0,X-SC,37.15,47852.0,-17154.0,168085.0,0.01,-26.39,351.26,232.18,198.0,-0.10,0.35,0.00,XY24,NTT,MISC
57,RELAXO,1176.00,-40.56,42.0,X-SC,3.71,81028.0,-64132.0,140057.0,-0.21,-44.18,172.85,52.31,136.0,-0.46,0.59,8.46,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-41.72,28.0,X-SC,8.73,87067.0,-41603.0,86902.0,0.20,-32.33,99.81,35.20,219.0,-0.48,0.63,0.20,X40,NTT,FOOTWEAR
2,ABBOTINDIA,35195.00,-10.45,46.0,X-MC,3.28,88500.0,-2178.0,17089.0,-0.17,-2.40,19.31,16.44,101.0,-0.13,0.64,17.29,X40,ATH,PHARMA
32,HONAUT,58357.33,-24.11,38.0,X-SC,0.93,106950.0,-20988.0,68127.0,0.41,-16.41,63.70,36.84,143.0,-0.31,0.77,9.49,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,-27.08,59.0,X-LC,9.00,290159.0,-55797.0,122505.0,1.31,-16.13,42.22,19.28,1.0,-0.46,2.10,7.27,X40,ATH,IT
39,INFY,2275.00,-15.29,67.0,X-LC,4.54,329809.0,16921.0,154779.0,1.18,5.41,46.93,54.87,3.0,0.11,2.39,14.14,X40,BTT,IT
41,ITC,452.00,-38.34,45.0,X-LC,0.83,198534.0,-1604.0,22494.0,-0.21,-0.80,11.33,10.44,4.0,-0.07,1.44,4.72,X40,NTT,FMCG
80,VBL,671.64,-17.63,50.0,X-LC,5.66,296861.0,-18981.0,131658.0,-1.13,-6.01,44.35,35.67,5.0,-0.14,2.15,7.01,X40N,ATH,FMCG
1,ABB,7934.00,-42.69,34.0,H-LC,5.00,239472.0,-22147.0,141360.0,0.21,-8.47,59.03,45.57,7.0,-0.16,1.73,1.90,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,COFFEEDAY,80.00,-47.69,46.0,L-SC,26.29,78221.0,-35328.0,75382.0,2.11,-31.11,96.37,35.27,268.0,-0.47,0.57,91.45,XR,NTT,HOTELS
7,ASIANTILES,137.00,7605.56,69.0,L-SC,15.03,84567.0,-9243.0,85861.0,0.71,-9.85,101.53,81.67,269.0,-0.11,0.61,63.77,XR,NTT,CERAMICS
49,MASFIN,398.61,-19.60,48.0,H-SC,7.87,91500.0,-6480.0,28081.0,0.84,-6.61,30.69,22.05,152.0,-0.23,0.66,33.00,XR,ATH,FINANCE
46,KPIGREEN,731.05,2.88,44.0,H-SC,9.78,122548.0,-2749.0,61678.0,0.41,-2.19,50.33,47.03,141.0,-0.04,0.89,53.44,MH,ATH,POWER
78,VAIBHAVGBL,521.00,69.15,58.0,H-SC,6.54,146781.0,-35994.0,147060.0,1.05,-19.69,100.19,60.77,125.0,-0.24,1.06,32.19,XR,NTT,JEWELLERY


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,ASIANTILES,137.00,7605.56,69.0,L-SC,15.03,84567.0,-9243.0,85861.0,0.71,-9.85,101.53,81.67,269.0,-0.11,0.61,63.77,XR,NTT,CERAMICS
78,VAIBHAVGBL,521.00,69.15,58.0,H-SC,6.54,146781.0,-35994.0,147060.0,1.05,-19.69,100.19,60.77,125.0,-0.24,1.06,32.19,XR,NTT,JEWELLERY
37,INDIGOPNTS,1408.00,124.89,74.0,M-SC,4.47,147349.0,-27210.0,27245.0,-1.19,-15.59,18.49,0.02,221.0,-1.00,1.07,28.17,OX40N,NTT,PAINTS
5,ANGELONE,3033.00,23.47,70.0,X-SC,6.87,218538.0,27532.0,30158.0,0.90,14.41,13.80,30.21,157.0,0.91,1.58,38.31,X40N,NTT,FINANCE
6,ASIANPAINT,3465.66,-7.57,73.0,X-LC,6.80,233350.0,-18418.0,71638.0,-0.21,-7.32,30.70,21.13,27.0,-0.26,1.69,25.52,X40,ATH,PAINTS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.54
1,25,45.97
2,50,77.50


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.99
LC    29.99
MC    25.01
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.91
X40      19.99
X40N     12.86
XY25     10.54
XR       10.08
AR        7.96
OX40N     6.80
X200      1.90
SR        1.06
MH        0.89
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.71
X-MC    22.25
X-LC    21.67
M-SC    12.97
X-SC     5.78
H-LC     4.73
M-LC     2.56
H-MC     2.41
L-SC     1.53
L-LC     1.03
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.47,-4.21,37.91
IT,13.20,-15.37,77.12
FINANCE,12.56,-8.77,58.07
MISC,7.94,-18.48,78.25
PAINTS,5.94,-12.51,29.29
ELECTRICAL,5.80,-12.57,52.48
INSURANCE,4.60,0.69,34.30
AUTO,2.88,-44.24,104.44
BANKS,2.83,-36.73,125.85


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3135297.0,20
XR,1264896.0,13
AR,1185985.0,9
X40,929442.0,13
X40N,691388.0,9
OX40N,544845.0,9
XY25,477739.0,7
SR,273748.0,2
X200,61815.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3414456.0,24
X-MC,1332727.0,15
M-SC,1250491.0,15
X-LC,1045932.0,12
X-SC,550532.0,6
H-LC,284665.0,3
H-MC,253733.0,2
L-SC,250393.0,3
M-LC,152373.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1181338.0      6
           AR         840028.0      5
           XR         798878.0      7
M-SC       XY24       727640.0      6
X-MC       XY24       568861.0      4
X-LC       X40        424569.0      5
X-MC       X40        417971.0      7
X-LC       XY24       308466.0      2
X-SC       X40N       295545.0      4
H-SC       SR         273748.0      2
           OX40N      258786.0      3
X-LC       X40N       236114.0      3
H-LC       AR         222850.0      2
X-MC       XY25       186166.0      2
H-MC       XY24       180907.0      1
X-SC       XY24       168085.0      1
L-SC       XR         161243.0      2
X-MC       X40N       159729.0      2
M-SC       XR         141638.0      2
           XY25       134023.0      1
           OX40N      124083.0      4
           AR         123107.0      2
M-LC       XR         108666.0      1
L-SC       OX40N       89150.0      1
X-SC       X40         86902.0      1
X-LC       XY25        76783.0      2
H-MC       OX40N       72826.0      1
H-LC       X200        61815.0      1
H-SC       MH          61678.0      1
L-MC       XR          54471.0      1
M-LC       XY25        43707.0      1
L-LC       XY25        37060.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
